# Compare Time Domain Survey and VASCA catalogs

In [1]:
# General path setup and library loading
%matplotlib widget
import matplotlib.pyplot as plt
from scipy.stats import chi2
import numpy as np

from jdaviz import Imviz
from astropy.coordinates import SkyCoord, match_coordinates_sky
from astropy import units as uu
from astropy.table import Table
from astropy.io import fits

from loguru import logger

from vasca.region import Region
from vasca.field import BaseField
import vasca.visualization as vvis

### Load VASCA region

In [2]:
region_name = "TDS_ELAISN1"  #_ELAISN1
region_fname = "./vasca_pipeline/"+region_name+"/region_"+region_name+".fits"
rg = Region()
rg.load_from_fits(region_fname)
display(rg.tt_fields)

field_id,name,ra,dec,observatory,obs_filter,fov_diam,sel,nr_vis,time_bin_size_sum,time_start,time_stop,rg_fd_id
,,deg,deg,,,deg,,,s,d,d,
bytes22,bytes22,float64,float64,bytes22,bytes22,float32,bool,int32,float32,float64,float64,int32
GNU2597312684324028416,PS_ELAISN1_MOS10,242.50638203682297,55.9801443588634,GALEX,NUV,1.2,True,24,29566.4,54960.17787037037,56821.33210648148,0
GNU2597383053068206080,PS_ELAISN1_MOS11,244.57200462478102,55.1825511621234,GALEX,NUV,1.2,True,25,31369.5,54960.24638888889,56815.50054271557,1
GNU2597453421812383744,PS_ELAISN1_MOS12,242.896209389035,55.0003489374844,GALEX,NUV,1.2,True,21,26349.451,54960.314884259256,56814.569061234084,2
GNU2597523790556561408,PS_ELAISN1_MOS13,241.28881797574297,55.3529870930308,GALEX,NUV,1.2,True,22,27810.902,54960.38337962963,56819.53759259259,3
GNU2597594159300739072,PS_ELAISN1_MOS14,243.95513153083402,54.1978605194106,GALEX,NUV,1.2,True,22,26881.852,54960.451886574076,56810.60611111111,4
GNU2597664528044916736,PS_ELAISN1_MOS15,242.396514585217,53.9999872930018,GALEX,NUV,1.2,True,22,28044.65,54960.52040509259,56810.67461805556,5
GNU2597734896789094400,PS_ELAISN1_MOS16,241.37385712300306,54.4519391954855,GALEX,NUV,1.2,True,21,26780.799,54960.588912037034,56807.7931854022,6


### Get TDS & VASCA data and match them positionally

In [3]:
#Select vasca sources, 5 sigma 0.000000573303, 4 sigma 0.00006334, 3 sigma = 0.002699796

sel_flux_cpval = (rg.tt_sources["flux_cpval"]<0.000000573303) * (rg.tt_sources["flux_cpval"]>-0.5)
sel_flux_nxv = rg.tt_sources["flux_nxv"]>0.0006
sel_flux_var = sel_flux_cpval*sel_flux_nxv

sel_assoc_ffactor = rg.tt_sources["assoc_ffactor"]>1.2
sel_assoc_fdiff_s2n = rg.tt_sources["assoc_fdiff_s2n"]>6
sel_assoc = sel_assoc_ffactor * sel_assoc_fdiff_s2n

sel_pos_cpval = rg.tt_sources["pos_cpval"]>1e-10
sel_flux = (rg.tt_sources["flux"]> 0.144543) * (rg.tt_sources["flux"]< 575.43)
sel_quality = sel_pos_cpval * sel_flux

sel_vasca = (sel_flux_var + sel_assoc )* sel_quality # 
rg.tt_sources["sel"][:]=sel_vasca

#sel_vasca = rg.tt_sources["sel"]
tt_vasca = rg.tt_sources[sel_vasca]

print(f"Selected {len(tt_vasca)} out of {len(rg.tt_sources)} sources")
# Print cut efficiencies
#def print_cuteff(new_sel,pre_sel,cut_label):
#    sel_temp = pre_sel*new_sel
#    print(f"Selection efficiency {cut_label}: {round(100*sel_temp.sum()/pre_sel.sum(),2)}%")
#print_cuteff(sel_flux_cpval,sel_flux_cpval*0+1,"flux_cpval")
#print_cuteff(sel_pos_cpval,sel_flux_cpval,"pos_cpval")
#print_cuteff(sel_flux_nxv,sel_pos_cpval*sel_flux_cpval,"nxv")

#Read TDS
tt_tds_all = Table.read("./TDS-ApJ_766_60/table4.fit", hdu="J_ApJ_766_60_table4")
ids = tt_tds_all["ID"].data.astype(dtype=str)
sel_elai = (np.char.find(ids, 'ELAI') > -1)
tt_tds = tt_tds_all[sel_elai] #tt_tds_all #
tt_tds['RAJ2000'].name = 'ra'
tt_tds['DEJ2000'].name = 'dec'

#Match VASCA-TDS
pos_tds = SkyCoord(ra=tt_tds["ra"], dec=tt_tds["dec"], unit="deg", frame="icrs")
pos_vasca = SkyCoord(ra=tt_vasca["ra"], dec=tt_vasca["dec"], unit="deg", frame="icrs")

match_dist = 10 * uu.arcsec
idx_tds, dist_tds, _ = pos_vasca.match_to_catalog_sky(pos_tds)
tt_vasca["tds_recno"]=tt_tds[idx_tds]["recno"]
tt_vasca["tds_idx"]=idx_tds
tt_vasca["tds_dist"]=dist_tds.to("arcsec")
tt_vasca["tds_match"] = tt_vasca["tds_dist"] < match_dist

idx_vasca, dist_vasca, _ = pos_tds.match_to_catalog_sky(pos_vasca)
tt_tds["rg_src_id"]=tt_vasca[idx_vasca]["rg_src_id"]
tt_tds["vasca_idx"]=idx_vasca
tt_tds["vasca_dist"]=dist_vasca.to("arcsec")
tt_tds["vasca_match"] = tt_tds["vasca_dist"] < match_dist

display(tt_vasca[:2])
display(tt_tds[:2])

Selected 594 out of 31322 sources


fd_src_id,nr_det,nr_uls,ra,dec,pos_err,pos_nxv,pos_var,pos_cpval,pos_rchiq,flux,flux_err,flux_nxv,flux_var,flux_cpval,flux_rchiq,flux_dmaxabs,flux_sigmax,flux_sigmax_dflux,flux_skew,mag,mag_err,assoc_id,assoc_dist,assoc_ffactor,assoc_fdiff_s2n,sel,rg_fd_id,rg_src_id,nr_fd_srcs,tds_recno,tds_idx,tds_dist,tds_match
,,,deg,deg,arcsec,,arcsec2,,,1e-06 Jy,1e-06 Jy,1e-12 Jy2,1e-12 Jy2,,,1e-06 Jy,,1e-06 Jy,1e-12 Jy2,mag,mag,,arcsec,,,,,,,,,arcsec,
int32,int32,int32,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int64,float32,float32,float32,bool,int32,int32,int32,int32,int64,float64,bool
-1,19,0,243.68769579044613,55.021948361962934,0.18757871,7.598095e-12,1.5080136,2.6854216e-06,2.2337022,8.027533,0.24923676,0.04459982,4.2858577,5.7080786e-08,3.85555,4.853364,3.981152,2.9207869,0.007890853,21.638546,0.033197403,-1,-1.0,-100.0,-10000.0,True,-1,123,2,630,99,379.2155586163366,False
-1,11,0,243.22666025870427,54.17832377430886,0.23234223,1.6014028e-12,1.6955272,1.3393981e-06,2.7161357,8.539153,0.31457403,0.06167135,5.7748156,3.6760787e-07,4.924303,4.1258698,3.3663406,3.0174532,-0.23716071,21.571463,0.03927803,23789,0.7691421,0.7828796,-5.461463,True,-1,256,2,965,151,373.34109780626255,False


recno,ID,ra,dec,l_mlow,mlow,l_Dmmax,Dmmax,l_sigint,sigint,l_Sd,Sd,l_Sy,Sy,LC,Mph,rmag,OT,z,X,Class,rg_src_id,vasca_idx,vasca_dist,vasca_match
,,deg,deg,,mag,,mag,,mag,,mag,,mag,,,mag,,,,,,,arcsec,
int32,bytes16,float64,float64,bytes1,float32,bytes1,float32,bytes1,float32,bytes1,float32,bytes1,float32,bytes1,bytes3,float32,bytes3,float32,bytes1,bytes12,int32,int64,float64,bool
3,ELAISN1_MOS15-02,242.0397,54.3586,>,22.89,>,4.03,,1.37,,0.84,,2.93,V,pt,22.51,QSO,--,,CV,940,10,41.4994,False
4,ELAISN1_MOS15-09,242.3685,53.6738,>,23.03,>,4.02,,1.23,,0.01,,2.15,F,ext,21.05,,--,,Galaxy Trans,27926,417,1.6707,True


In [4]:
sel_tmv = tt_tds["vasca_match"]
sel_vmt = tt_vasca["tds_match"]
print("Nr. sources VASCA/TDS :",len(tt_vasca),"/",len(tt_tds))
print("Nr. associated sources VASCA/TDS:",sel_vmt.sum(),"/",sel_tmv.sum())
print("Fraction of associated VASCA/TDS sources:",sel_vmt.sum()/len(tt_vasca),"/",sel_tmv.sum()/len(tt_tds))

ids_tds = tt_tds["ID"].data.astype(dtype=str)
sel_tds_mos15 = (np.char.find(ids_tds, 'MOS15') > -1)
tt_tds_mos15 = tt_tds[sel_tds_mos15]
print("Total Nr of TDS sources from MOS15:",len(tt_tds_mos15))
display(tt_tds_mos15[tt_tds_mos15["vasca_match"]==False]["recno","ID","ra","dec"])

Nr. sources VASCA/TDS : 594 / 160
Nr. associated sources VASCA/TDS: 81 / 82
Fraction of associated VASCA/TDS sources: 0.13636363636363635 / 0.5125
Total Nr of TDS sources from MOS15: 26


recno,ID,ra,dec
,,deg,deg
int32,bytes16,float64,float64
3,ELAISN1_MOS15-02,242.0397,54.3586
59,ELAISN1_MOS15-10,242.3832,53.6801
71,ELAISN1_MOS15-20,242.8292,53.7492
127,ELAISN1_MOS15-16,242.7175,53.7547
184,ELAISN1_MOS15-24,243.0512,54.2599
212,ELAISN1_MOS15-11,242.4293,54.3621
307,ELAISN1_MOS15-03,242.1960,54.0721
390,ELAISN1_MOS15-22,242.8843,53.5921


In [ ]:
#PS_ELAISN1_MOS15 is field_id GNU2597664528044916736 (TDE PS1-10jh field)
field_id = rg.tt_fields["field_id"][0]
fd = rg.get_field(field_id=field_id, load_method="FITS", add_field=False)

#from astropy.coordinates import Angle, SkyCoord
#from regions import CircleSkyRegion
#center = SkyCoord(243.0512, 54.2599, unit='deg')
#radius = Angle(0.03, 'deg')
#region = CircleSkyRegion(center, radius)

fig, ax = vvis.plot_field_sky_map(fd, img_idx=-1, sky_region = fd.get_sky_region()) # 
ax = vvis.plot_sky_sources(rg.tt_sources,tt_det=rg.tt_detections, src_id="rg_src_id", sky_region_wcs = (fd.get_sky_region(),fd.ref_wcs))

plot_tds = {"marker":"x","color":"k"}
ax = vvis.plot_sky_sources(tt_tds, src_id="recno",src_kwargs=plot_tds, sky_region_wcs = (fd.get_sky_region(),fd.ref_wcs))

In [ ]:
import vasca.visualization as vvis
_= vvis.plot_pipe_diagnostic(rg, "tt_sources", "scatter")
_= vvis.plot_pipe_diagnostic(rg, "tt_sources", "hist")
_= vvis.plot_pipe_diagnostic(fd, "tt_detections", "hist")
_= vvis.plot_pipe_diagnostic(fd, "tt_detections", "scatter")

In [ ]:
#rg.tt_coadd_sources["sel"][:]=rg.tt_coadd_sources["flux"]/rg.tt_coadd_sources["flux_err"]>5
display(rg.tt_coadd_sources)
_ = vvis.plot_pipe_diagnostic(rg, "tt_coadd_sources", plot_type="hist", fig_size=(16, 4))
_ = vvis.plot_pipe_diagnostic(rg, "tt_coadd_sources", plot_type="scatter", fig_size=(12, 4))